In [1]:
import sys
print(sys.executable)

/mnt/d2ddf6ba-3ea2-45b7-a8f7-2760a03a911a/snake-multi/.venv/bin/python3


In [2]:
import os
os.chdir("../..")
os.getcwd()

'/mnt/d2ddf6ba-3ea2-45b7-a8f7-2760a03a911a/snake-multi'

In [3]:
import random
import logging
import sys
from collections import deque
from functools import partial

import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from src.simulation.simulation import Simulation, Vector2
from src.simulation.board_generator import make_simple_board

from src.model.model_utils import get_model_moves, get_model_scores, get_rotated_state

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)

2025-09-01 14:22:32.337441: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-01 14:22:32.345521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756729352.354136 1617802 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756729352.356774 1617802 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756729352.363810 1617802 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
view_range = 7
replay_buffer_size = 50000

In [5]:
def calculate_score(ate_food: bool, died: bool):
    if died:
        return -10

    if ate_food:
        return 10

    return 0

In [6]:
cat_encoding = tf.keras.layers.CategoryEncoding(num_tokens=6, output_mode="one_hot")

cat_encoding([[3, 2, 1, 5], [5, 3, 1, 3]])

I0000 00:00:1756729353.625539 1617802 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8027 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


<tf.Tensor: shape=(2, 4, 6), dtype=float32, numpy=
array([[[0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.]]], dtype=float32)>

In [7]:
#online_model = tf.keras.Sequential([
#    tf.keras.layers.Input((view_range * 2 + 1, view_range * 2 + 1)),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.CategoryEncoding(num_tokens=6, output_mode="one_hot"),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(42, activation="relu", kernel_initializer="he_normal"),
#    tf.keras.layers.Dense(42, activation="relu", kernel_initializer="he_normal"),
#    tf.keras.layers.Dense(1)
#])

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal")

online_model = tf.keras.Sequential([
    tf.keras.layers.Input((view_range * 2 + 1, view_range * 2 + 1)),
    tf.keras.layers.CategoryEncoding(num_tokens=7, output_mode="one_hot"),
    DefaultConv2D(filters=32, kernel_size=7),
    DefaultConv2D(filters=32, kernel_size=5),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1)
])

target_model = tf.keras.models.clone_model(online_model)
target_model.set_weights(online_model.get_weights())

convolutional = True

In [8]:
model_name = "r7_conv_32-32_64-64_d_32-32_lr01"

In [9]:
logging.basicConfig(level=logging.DEBUG)

log_formatter = logging.Formatter("%(message)s")

std_handler = logging.StreamHandler(sys.stdout)
std_handler.setLevel(logging.INFO)
std_handler.setFormatter(log_formatter)

file_handler = logging.FileHandler(f"logs_{model_name}.log")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(log_formatter)

logger = logging.getLogger()
logger.handlers.clear()
#logger.addHandler(std_handler)
logger.addHandler(file_handler)

In [10]:
online_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ category_encoding_1             │ (None, 15, 15, 7)      │             0 │
│ (CategoryEncoding)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 15, 15, 32)     │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 15, 15, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,233 (372.00 KB)

 Trainable params: 95,233 (372.00 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
simulation = Simulation(calculate_score)
simulation.reset(make_simple_board(np.array([view_range * 2 + 1, view_range * 2 + 1])), 2, 1)

online_model.predict(np.array([simulation.get_snake_view(0, convolutional, view_range)]))

I0000 00:00:1756729354.099297 1626411 service.cc:152] XLA service 0x737c380156a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756729354.099313 1626411 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-09-01 14:22:34.103996: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756729354.122201 1626411 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step


I0000 00:00:1756729354.541051 1626411 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[-0.09491814]], dtype=float32)

In [12]:
simulation.get_snake_view(0, True)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [13]:
tf.keras.layers.CategoryEncoding(num_tokens=7, output_mode="one_hot")([simulation.get_snake_view(0, True)])

<tf.Tensor: shape=(1, 15, 15, 7), dtype=float32, numpy=
array([[[[0., 1., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         ...,
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.]],

        [[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.]],

        [[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0

In [14]:
def epsilon_greedy(snakes_states: list[np.ndarray], snakes_possible_moves: list[list[Vector2]], snakes_alive: list[bool], epsilon: float=0.):
    n_snakes = len(snakes_states)

    process_mask = [True for _ in range(n_snakes)]
    
    for i, alive in enumerate(snakes_alive):
        if not alive or np.random.rand() < epsilon:
            process_mask[i] = False

    moves = get_model_moves(online_model, snakes_states, snakes_possible_moves, process_mask, softmax=True)

    for i, move in enumerate(moves):
        if move == (0, 0):
            moves[i] = random.choice(simulation.get_legal_moves(i))

    return moves

In [15]:
replay_buffer = deque(maxlen=replay_buffer_size)

In [16]:
def play_one_step(simulation: Simulation, states_before: list[np.ndarray], possible_moves_before: list[Vector2], snakes_alive_before: list[bool], epsilon: float):
    # Save states
    moves = epsilon_greedy(states_before, possible_moves_before, snakes_alive_before, epsilon)

    scores, running = simulation.next(moves)

    states_after = [simulation.get_snake_view(i, convolutional, view_range) for i in range(simulation.n_snakes)]
    possible_moves_after = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive_after = simulation.snakes_alive

    for i in range(len(states_before)):
        if snakes_alive_before[i]:
            replay_buffer.append((states_before[i], moves[i], scores[i], states_after[i], possible_moves_after[i], snakes_alive_after[i]))

    return states_after, possible_moves_after, snakes_alive_after, scores, running

In [17]:
def sample_experiences(batch_size: int):
    idxs = np.random.randint(len(replay_buffer), size=batch_size)
    
    batch = [replay_buffer[idx] for idx in idxs]

    return [
        [experience[field_idx] for experience in batch]
        for field_idx in range(6)
    ]

In [18]:
batch_size = 32
discount_factor = 0.9
loss_fn = tf.keras.losses.mse
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

losses = []


def training_step():
    experiences = sample_experiences(batch_size)
    states, moves, scores, states_after, possible_moves_after, snakes_alive_after = experiences

    best_next_moves = [[move] for move in get_model_moves(online_model, states_after, possible_moves_after, snakes_alive_after)]
    next_q_values = get_model_scores(target_model, states_after, best_next_moves, snakes_alive_after)
    
    #next_q_values = get_model_scores(target_model, states_after, possible_moves_after, snakes_alive_after)
    runs = 1.0 - np.array(snakes_alive_after)
    
    target_q_values = scores + runs * discount_factor * next_q_values

    X = tf.constant([get_rotated_state(state, move) for state, move in zip(states, moves)])
    
    with tf.GradientTape() as tape:
        predicted_q_values = online_model(X)
        loss = tf.reduce_mean(loss_fn(target_q_values, predicted_q_values))

    losses.append(loss.numpy())
    grads = tape.gradient(loss, online_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, online_model.trainable_variables))

In [ ]:
simulation = Simulation(calculate_score)
rewards = []
steps = []
replay_buffer.clear()

best_model_window = 14
best_model_rewards = float("-inf")

n_episodes = 100000

for episode in range(n_episodes):
    simulation.reset(make_simple_board(np.array([15, 15])), snake_count=1, food_count=2)

    rewards.append(0)

    states = [simulation.get_snake_view(i, convolutional, view_range) for i in range(simulation.n_snakes)]
    possible_moves = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive = simulation.snakes_alive

    for step in range(200):
        epsilon = max(1 - (episode / (n_episodes * 0.8)), 0.005)
        #discount_factor = 0.95 * (episode / n_episodes)
        
        states, possible_moves, snakes_alive, scores, running = play_one_step(simulation, states, possible_moves, snakes_alive, epsilon)

        rewards[-1] += sum(scores)

        if not running:
            break

    steps.append(step)

    logging.info(f"Episode {episode} played")
    logging.info(f"\tSteps: {step}")
    logging.info(f"\tRewards: {rewards[-1]}")

    if episode > 200:
        curr_model_rewards = sum(rewards[-best_model_window:])
        if curr_model_rewards > best_model_rewards:
            avg_rewards = curr_model_rewards / best_model_window
            online_model.save(f"models/{model_name}_{round(avg_rewards)}_snapshot.keras")
            best_model_rewards = curr_model_rewards
            logging.info(f"\tSnaphshot saved (avg. {avg_rewards})")

        training_step()
        logging.info("\tTraining finished")
        
        if episode % 100 == 0:
            target_model.set_weights(online_model.get_weights())
            logging.info("\tTarget model updated")

In [ ]:
#sns.lineplot(steps, label="steps")
fig, ax = plt.subplots(figsize=(16,6))
sns.lineplot(np.convolve(rewards, [1 for _ in range(10)]))
plt.xlabel("Episode")
plt.ylabel("Reward")

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
sns.lineplot(np.convolve(losses, [1 for _ in range(10)]), ax=ax)
plt.xlabel("Episode")
plt.ylabel("Loss")

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
sns.lineplot(np.cumsum(rewards), label="rewards", ax=ax)
plt.xlabel("Episode")
plt.ylabel("Reward")

In [ ]:
online_model.save(f"models/{model_name}.keras")